# Yahoofinance API: Historical Data

This code is used to collect data concerning prices of different investment instruments over a specified period of time and to load the results into a PostgreSQL database. It is a one-time action to obtain the required data from the past.

In [ ]:
# import packages
from yfapi import YahooFinanceAPI, Interval
from datetime import datetime, timedelta
import psycopg2
from sqlalchemy import create_engine
from decouple import config

In [ ]:
# Get database credentials, db path and twitter bearer token from env-file
dbuname = config('DBUNAME')
dbpwd = config('DBPWD')
host = config('HOST')
port = config('PORT')
dbname = config('DBNAME')

In [ ]:
# instantiate the API and set the interval to daily data
dh = YahooFinanceAPI(Interval.DAILY)

In [ ]:
# define function for fetching and loading the data

def fetch_load_data(instrument, label, start, end, method):

    while True:
        #fetch data
        try:
            df = dh.get_ticker_data(instrument, start, end)
            table_name = 'table_yf_'+label
            print(table_name)
            df.rename(columns={'Date': 'date'}, inplace=True)
            df['Ins'] = instrument
            df['Ins_label'] = label
        except Exception as e:
            print(e)
            print("Investment Instrument not available: " + instrument)
            break


        #load data
        if_ex_val = method

        try:
            conn_string = 'postgresql://'+dbuname+':'+dbpwd+'@'+host+':'+port+'/'+dbname
            engine = create_engine(conn_string)
            df.to_sql(table_name, conn_string, if_exists = if_ex_val, index=False)
        except Exception as e:
            print(e)
            print("Data load failed: " + table_name)
            break
        break


In [ ]:
# Initial load of the historical data

def main():

    # Investment instruments
    inv_instruments = ['URTH', 'EXW1.DE', 'CSSMI.SW', 'EXXT.DE', 'GC=F', 'SI=F']
    labels={'GC=F':'gold','SI=F':'silver','URTH':'msci_world', 'EXW1.DE':'euro_stoxx', 'CSSMI.SW':'smi', 'EXXT.DE':'nasdaq'}

    # set timeframe
    now = datetime(2022, 5, 6)
    then = datetime(2021, 1, 1)

    # set method of function df.to_sql()
    method = 'replace'

    # loop for all investment instruments
    for ins in inv_instruments:
        label = labels[ins]
        fetch_load_data(ins, label, then, now, method)

In [ ]:
# execute function
if __name__ == "__main__":
    main()

table_yf_msci_world
table_yf_euro_stoxx
table_yf_smi
table_yf_nasdaq
table_yf_gold
table_yf_silver
